In [10]:
import minsearch
import json

In [11]:
with open("documents.json", "rt")as f_in:
    doc_raw = json.load(f_in)

In [12]:
documents = []


for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [13]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
                )

In [ ]:
select * where course = 'data-engineering-zoomcamp';

In [20]:
q = 'the course has already started, can I still enroll?'

In [21]:
index.fit(documents)

In [26]:
boost = {'question':3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={"course":"data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
    
)

In [27]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [7]:
from openai import OpenAI
import os

In [14]:
client = OpenAI()


In [16]:
q = "i just discovered the course, can i still join?"

In [17]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role":"user", "content": q}]
)

In [37]:
response.choices[0].message.content

"It's best to check with the specific institution or organization offering the course. Many courses have a late enrollment period and some may allow you to join even after the course has started, possibly with some catch-up requirements. Contact the course administrator or registrar to inquire about your specific situation and what steps you would need to take to enroll late."

In [56]:
prompt_template = """
You are course teaching assistant. 
Answer the QUESTION based on the CONTEXT from FAQ DATABASE. 
Use only the facts from the CONTEXT when answering the QUESTION
if the CONTEXT doesn't contain the answer, output NONE

QUESTION : {question}
CONTEXT:
{context}
""".strip()

In [54]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [60]:
prompt = prompt_template.format(question=q,context=context).strip()

In [61]:
print(prompt)

You are course teaching assistant. 
Answer the QUESTION based on the CONTEXT from FAQ DATABASE. 
Use only the facts from the CONTEXT when answering the QUESTION
if the CONTEXT doesn't contain the answer, output NONE

QUESTION : the course has already started, can I still enroll?
CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final c

In [62]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role":"user", "content": prompt}]
)

In [64]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."

In [97]:
def search(query):
    boost = {'question':3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={"course":"data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5
        
    )
    return results

In [18]:
def build_prompt(query, search_results):
    
    prompt_template = """
    You are course teaching assistant. 
    Answer the QUESTION based on the CONTEXT from FAQ DATABASE. 
    Use only the facts from the CONTEXT when answering the QUESTION
    
    QUESTION : {question}
    CONTEXT:
    {context}
    """.strip()
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
   
    prompt = prompt_template.format(question=query,context=context).strip()

    return prompt

In [19]:
def llm(prompt):
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role":"user", "content": prompt}]
    )
    return response.choices[0].message.content

In [34]:
def rag(query):
    results = elastic_search(query)
    prompt = build_prompt(query, results)
    answer = llm(prompt)
    return answer

In [100]:
rag('the course already started can i still enroll?')

'Yes, even if the course has already started, you are still eligible to enroll and submit the homeworks. However, please be mindful of the deadlines for turning in the final projects.'

In [5]:
documents[0] 

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [28]:
from elasticsearch import Elasticsearch

In [29]:
es_client = Elasticsearch('http://localhost:9200')

In [33]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-2'})

In [15]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████| 948/948 [00:25<00:00, 37.55it/s]


In [21]:
query = "I just discovered the course can i still join?"

In [26]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    results_docs = []
    for hit in response['hits']['hits']:
        results_docs.append(hit['_source'])
    return results_docs

In [37]:
index_name=''
elastic_search(query)

[]